In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=a25f28a9da08c6e0de494314f5da736f98a24110bbe0a4f37ab1129b1f1c031c
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [5]:
from pyspark.sql  import SparkSession
from pyspark.sql import SQLContext

if __name__ == '__main__':
  scSpark=SparkSession.builder.appName("reading csv").getOrCreate()

In [8]:
datafile='/content/sample_data/california_housing_test.csv'
sdfdata=scSpark.read.csv(datafile,header=True,sep=',').cache()
print('Total records = {}'.format(sdfdata.count()))
sdfdata.show()

super='/content/supermarket.csv'
supermarket=scSpark.read.csv(super,header=True,sep=',') .cache()
gender=supermarket.groupBy('Gender').count()
print(gender.show())

Total records = 3000
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|-122.050000|37.370000|         27.000000|3885.000000|    661.000000|1537.000000| 606.000000|     6.608500|     344700.000000|
|-118.300000|34.260000|         43.000000|1510.000000|    310.000000| 809.000000| 277.000000|     3.599000|     176500.000000|
|-117.810000|33.780000|         27.000000|3589.000000|    507.000000|1484.000000| 495.000000|     5.793400|     270500.000000|
|-118.360000|33.820000|         28.000000|  67.000000|     15.000000|  49.000000|  11.000000|     6.135900|     330000.000000|
|-119.670000|36.330000|         19.000000|1241.000000|    244.000000| 850.000000| 237.0000

In [14]:
supermarket.registerTempTable("market")
output=scSpark.sql("select * from market where `UNIT PRICE` < 15 AND QUANTITY < 10")
output.show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+------+--------+---------+-----+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity|Tax 5%|   Total|     Date| Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|351-62-0822|     B| Mandalay|       Member|Female| Fashion accessories|     14.48|       4| 2.896|  60.816| 2/6/2019|18:07|    Ewallet| 57.92|            4.761904762|       2.896|   4.5|
|871-39-9221|     C|Naypyitaw|       Normal|Female|Electronic access...|     12.45|       6| 3.735|  78.435| 2/9/2019|13:11|       Cash|  74.7|            4.761904762|       3.735|   4.1|
|586-25-0848|     A|   Yangon|       Normal|Female|   Sports

In [15]:
output.write.format('json').save("data.json")